In [19]:
import os
import sys
import pandas as pd
from pyspark.sql import SparkSession
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

In [20]:
re1=pd.read_csv('data/reviews_0-250.csv')
re2=pd.read_csv('data/reviews_250-500.csv')
re3=pd.read_csv('data/reviews_500-750.csv')
re4=pd.read_csv('data/reviews_750-1250.csv')
re5=pd.read_csv('data/reviews_1250-end.csv')

C:\Users\marti\AppData\Local\Temp\ipykernel_21952\99614358.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  re1=pd.read_csv('data/reviews_0-250.csv')
C:\Users\marti\AppData\Local\Temp\ipykernel_21952\99614358.py:4: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  re4=pd.read_csv('data/reviews_750-1250.csv')
C:\Users\marti\AppData\Local\Temp\ipykernel_21952\99614358.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  re5=pd.read_csv('data/reviews_1250-end.csv')


In [21]:
df=pd.concat([re1, re2, re3, re4, re5], ignore_index=True)

In [22]:
df.shape

(1094411, 19)

In [23]:
df.author_id.nunique()

578653

In [24]:
df1=df.copy()

In [25]:
author_counts = df['author_id'].value_counts()

# Filtrar los author_id que se repiten al menos 3 veces
author_to_keep = author_counts[author_counts >= 5].index

# Actualizar el DataFrame original para conservar solo los author_id que se repiten al menos 3 veces
df = df[df['author_id'].isin(author_to_keep)]

In [26]:
df = df[['author_id', 'rating', 'product_id']]

In [27]:
df.shape

(291493, 3)

In [28]:
df['author_id'] = pd.factorize(df['author_id'])[0] + 1


In [29]:
df.head()

,author_id,rating,product_id
29,1,4,P420652
86,2,5,P420652
115,3,3,P420652
192,4,1,P420652
194,5,4,P420652


In [30]:
df.dtypes

author_id      int64
rating         int64
product_id    object
dtype: object

In [31]:
df['author_id'] = df['author_id'].astype(str)

# Filter out rows where author_id starts with 'order'
df = df[~df['author_id'].str.startswith(('order'))]
df['product_id'] = df['product_id'].apply(lambda x: x[1:])

In [32]:

df['author_id'] = df['author_id'].astype('int64')
df['product_id'] = df['product_id'].astype('int64')


In [33]:
df.dtypes

author_id     int64
rating        int64
product_id    int64
dtype: object

In [34]:
df.to_csv("data/df_als.csv")

In [35]:
#!pip install pyspark

In [36]:

from pyspark.sql.functions import col, explode
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

In [ ]:

num_ratings = df["rating"].count()
num_users = df["author_id"].nunique()
num_movies = df["product_id"].nunique()

denominator = num_users * num_movies

sparsity = (1.0 - (num_ratings * 1.0) / denominator) * 100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  99.56% empty.


In [ ]:
from implicit.als import AlternatingLeastSquares
model = AlternatingLeastSquares(factors=190, regularization=0.01, iterations=10)

c:\Users\Nicolas\AppData\Local\Programs\Python\Python310\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


## Spark

In [ ]:
!pip install pyspark

You should consider upgrading via the 'C:\Users\Nicolas\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [ ]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

In [ ]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
df_spark = spark.createDataFrame(df)

In [ ]:
# Create test and train set
(train, test) = df_spark.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(rank=150, regParam=0.01, userCol="author_id", itemCol="product_id", ratingCol="rating",coldStartStrategy="drop")
# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

In [ ]:


# Entrenar el modelo ALS
model = als.fit(train)


In [ ]:
# Guardar el modelo
model.save("modelo_als")


<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 150
  MaxIter: 10
  RegParam: 0.01

In [ ]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error = {rmse}")

Root-mean-square error = 0.9732130069092246


In [ ]:
nrecommendations = model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+---------+--------------------+
|author_id|     recommendations|
+---------+--------------------+
|        1|[{478030, 4.99902...|
|       12|[{420652, 4.99998...|
|       13|[{469088, 5.00474...|
|       22|[{422905, 5.11993...|
|       26|[{420652, 4.99070...|
|       27|[{427406, 5.24030...|
|       28|[{469088, 5.80673...|
|       31|[{173726, 5.44013...|
|       34|[{407444, 4.99992...|
|       44|[{302103, 5.36987...|
+---------+--------------------+



In [ ]:
from pyspark.sql.functions import col, explode

# Explode the recommendations column
nrecommendations = nrecommendations.withColumn("rec_exp", explode("recommendations"))

# Select the relevant columns
nrecommendations = nrecommendations.select('author_id', col("rec_exp.product_id"), col("rec_exp.rating"))

# Show the first 10 rows
nrecommendations.limit(10).show()


+---------+----------+---------+
|author_id|product_id|   rating|
+---------+----------+---------+
|        1|    478030|4.9990263|
|        1|    471086|  4.99266|
|        1|    464288|  4.99266|
|        1|    427406| 4.698785|
|        1|    454102|4.4097824|
|        1|    455364| 4.375922|
|        1|    173726| 4.348395|
|        1|    433887|4.3283653|
|        1|    422905|4.3254585|
|        1|    302103|4.3204165|
+---------+----------+---------+



In [ ]:
from pyspark.sql.functions import col, substring
product= spark.read.csv("data/product_info.csv",header=True)
product = product.withColumn('product_id', substring(col('product_id'), 2, 100))

In [ ]:
nrecommendations.join(product, on='product_id').filter('author_id=44').show()

+----------+---------+---------+--------------------+--------+-----------------+-----------+------+-------+---------------+--------------+---------------+--------------------+--------------------+---------+---------------+--------------+---------------+---+-----------+------------+-----------------+--------------------+----------------+--------------------+-----------------+-----------+---------------+---------------+
|product_id|author_id|   rating|        product_name|brand_id|       brand_name|loves_count|rating|reviews|           size|variation_type|variation_value|      variation_desc|         ingredients|price_usd|value_price_usd|sale_price_usd|limited_edition|new|online_only|out_of_stock|sephora_exclusive|          highlights|primary_category|  secondary_category|tertiary_category|child_count|child_max_price|child_min_price|
+----------+---------+---------+--------------------+--------+-----------------+-----------+------+-------+---------------+--------------+---------------+--

In [ ]:
# Joining the DataFrames on 'product_id'
joined_df = df_spark.join(product, df_spark['product_id'] == product['product_id'])

filtered_df = joined_df.filter(col('author_id') == 44)
filtered_df.show()


+---------+------+----------+----------+--------------------+--------+--------------+-----------+------+-------+--------------+--------------+---------------+--------------+--------------------+---------+---------------+--------------+---------------+---+-----------+------------+-----------------+--------------------+----------------+--------------------+--------------------+-----------+---------------+---------------+
|author_id|rating|product_id|product_id|        product_name|brand_id|    brand_name|loves_count|rating|reviews|          size|variation_type|variation_value|variation_desc|         ingredients|price_usd|value_price_usd|sale_price_usd|limited_edition|new|online_only|out_of_stock|sephora_exclusive|          highlights|primary_category|  secondary_category|   tertiary_category|child_count|child_max_price|child_min_price|
+---------+------+----------+----------+--------------------+--------+--------------+-----------+------+-------+--------------+--------------+------------